# Data Cleaning
### Data cleaning is a key part of data science, but it can be deeply frustrating.
•	**Why are some of your text fields garbled?**             
•	**What should you do about those missing values?**             
•	**Why aren’t your dates formatted correctly?**               
•	**How can you quickly clean up inconsistent data entry?**                 

### Note: In this course, you'll learn why you've run into these problems and, more importantly, how to fix them!
•	In this course, you’ll learn how to tackle some of the most common data cleaning problems so you can get to actually analyzing your data faster. You’ll work through five hands-on exercises with real, messy data and answer some of your most commonly-asked data cleaning questions.




**Step 1: Take a first look at Data**

In [3]:
# modules we'll use

import pandas as pd
import numpy as np

In [120]:
!pip install nbconvert


[notice] A new release of pip is available: 23.0.1 -> 23.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
# read in all our data

fb_event_data = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")

C:\Users\kumar\AppData\Local\Temp/ipykernel_18332/498409280.py:3: DtypeWarning: Columns (25,51) have mixed types. Specify dtype option on import or set low_memory=False.
  fb_event_data = pd.read_csv("NFL Play by Play 2009-2016 (v3).csv")


### **The first thing to do when you get a new dataset is take a look at some of it. This lets you see that it all read in correctly and gives an idea of what's going on with the data. In this case, let's see if there are any missing values, which will be reprsented with NaN or None.**

In [9]:
# Look some data is good way using head()
# I can see a handful of missing data already
fb_event_data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


#### Yep, it looks like there's some missing values.

## How many missing data points do we have?

- Ok, Now we know that we do have some missing values. Let's see how many we have in each **column**

In [18]:
# get the number of missing data points per column

missing_values_count = fb_event_data.isnull().sum() # isnull() look for null values at every column one by one

# look at the missing points in the first ten columns but we have 102 columns in our dataset

missing_values_count[0:10]

Date                0
GameID              0
Drive               0
qtr                 0
down            54218
time              188
TimeUnder           0
TimeSecs          188
PlayTimeDiff      374
SideofField       450
dtype: int64



**That seems like a lot! It might be helpful to see what percentage of the values in our dataset were missing to give us a better sense of the scale of this problem:**

In [25]:
# how many total missing values do we have?

total_cells = np.product(fb_event_data.shape) # this is total number of cell
total_missing = missing_values_count.sum() # this is total number of missing values

# percent of data that is missing
percent_missing = round((total_missing / total_cells)* 100,2)
print(f"{percent_missing} %")

27.65 %


**Wow, almost a quarter of the cells in this dataset are empty! In the next step, we're going to take a closer look at some of the columns with missing values and try to figure out what might be going on with them.**

In [31]:
# Below code is example for the above to explain what happen in the above code

In [26]:
data = [[1,2,3],[4,4,5],[6,4,3]]

In [29]:
df = pd.DataFrame(data)

In [33]:
df.shape

(3, 3)

In [32]:
np.product(df.shape)


9

## Figure out why the data is missing

This is the point at which we get into the part of data science that I like to call "data intution", by which I mean "really looking at your data and trying to figure out why it is the way it is and how that will affect your analysis". It can be a frustrating part of data science, especially if you're newer to the field and don't have a lot of experience. For dealing with missing values, you'll need to use your intution to figure out why the value is missing. One of the most important questions you can ask yourself to help figure this out is this:

### - Is this value missing because it wasn't recorded or because it doesn't exist?



If a value is missing becuase it doesn't exist (like the height of the oldest child of someone who doesn't have any children) then it doesn't make sense to try and guess what it might be. These values you probably do want to keep as NaN. On the other hand, **if a value is missing because it wasn't recorded, then you can try to guess what it might have been based on the other values in that column and row. This is called imputation(the process of replacing missing data with substituted values), and we'll learn how to do it next! :)**


Let's work through an example. Looking at the number of missing values in the nfl_data dataframe, I notice that the column "TimesSec" has a lot of missing values in it:




In [40]:
# look at the # of missing points in the first ten columns

missing_values_count[50:100]

Returner                       339535
BlockingPlayer                 362341
Tackler1                       147183
Tackler2                       318045
FieldGoalResult                354431
FieldGoalDistance              354528
Fumble                              0
RecFumbTeam                    358513
RecFumbPlayer                  358513
Sack                                0
Challenge.Replay                    0
ChalReplayResult               359476
Accepted.Penalty                    0
PenalizedTeam                  336362
PenaltyType                    347849
PenalizedPlayer                337483
Penalty.Yards                       0
PosTeamScore                    23682
DefTeamScore                    23682
ScoreDiff                       21975
AbsScoreDiff                    23682
HomeTeam                            0
AwayTeam                            0
Timeout_Indicator                   0
Timeout_Team                   347538
posteam_timeouts_pre                0
HomeTimeouts

By looking at the documentation, I can see that this column has information on the number of seconds left in the game when the play was made. This means that these values are probably missing because they were not recorded, rather than because they don't exist. So, it would make sense for us to try and guess what they should be rather than just leaving them as NA's.

On the other hand, there are other fields, like "PenalizedTeam" that also have lot of missing fields. In this case, though, the field is missing because if there was no penalty then it doesn't make sense to say which team was penalized. For this column, it would make more sense to either leave it empty or to add a third value like "neither" and use that to replace the NA's.

Tip: This is a great place to read over the dataset documentation if you haven't already! If you're working with a dataset that you've gotten from another person, you can also try reaching out to them to get more information.

If you're doing very careful data analysis, this is the point at which you'd look at each column individually to figure out the best strategy for filling those missing values. For the rest of this notebook, we'll cover some "quick and dirty" techniques that can help you with missing values but will probably also end up removing some useful information or adding some noise to your data.

# Drop missing values
If you're in a hurry or don't have a reason to figure out why your values are missing, one option you have is to just remove any rows or columns that contain missing values. (Note: I don't generally recommend this approch for important projects! It's usually worth it to take the time to go through your data and really look at all the columns with missing values one-by-one to really get to know your dataset.)

If you're sure you want to drop rows with missing values, pandas does have a handy function, dropna() to help you do this. Let's try it out on our NFL dataset!



In [41]:
new_event_data = fb_event_data.dropna() 

In [43]:
new_event_data

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season


In [45]:
fb_event_data.head()

,Date,GameID,Drive,qtr,down,time,TimeUnder,TimeSecs,PlayTimeDiff,SideofField,...,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2009-09-10,2009091000,1,1,NaN,15:00,15,3600.0,0.0,TEN,...,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,2009-09-10,2009091000,1,1,1.0,14:53,15,3593.0,7.0,PIT,...,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,2009-09-10,2009091000,1,1,2.0,14:16,15,3556.0,37.0,PIT,...,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,2009-09-10,2009091000,1,1,3.0,13:35,14,3515.0,41.0,PIT,...,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2009-09-10,2009091000,1,1,4.0,13:27,14,3507.0,8.0,PIT,...,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009


In [53]:

columns_with_na_dropped = fb_event_data.dropna(axis = 1)
columns_with_na_dropped.head()

,Date,GameID,Drive,qtr,TimeUnder,ydstogo,ydsnet,PlayAttempted,Yards.Gained,sp,...,AwayTeam,Timeout_Indicator,posteam_timeouts_pre,HomeTimeouts_Remaining_Pre,AwayTimeouts_Remaining_Pre,HomeTimeouts_Remaining_Post,AwayTimeouts_Remaining_Post,ExPoint_Prob,TwoPoint_Prob,Season
0,2009-09-10,2009091000,1,1,15,0,0,1,39,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
1,2009-09-10,2009091000,1,1,15,10,5,1,5,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
2,2009-09-10,2009091000,1,1,15,5,2,1,-3,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
3,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009
4,2009-09-10,2009091000,1,1,14,8,2,1,0,0,...,TEN,0,3,3,3,3,3,0.0,0.0,2009


In [48]:
# just how much data did we lose 
print("Columns in original dataset: %d \n" % fb_event_data.shape[1])
print("Columns with na's Dropped: %d" % columns_with_na_dropped.shape[1])

Columns in original dataset: 102 

Columns with na's Dropped: 37


**Example to understand the concept of axis in pandas**

In [95]:
matrixA = [[1,None,3],[4,5,None],[7,8,9]]
name_c = ('X','Y','Z')

In [96]:
df_m = pd.DataFrame(matrixA, columns=name_c)

In [102]:
df_m

,X,Y,Z
0,1,NaN,3.0
1,4,5.0,NaN
2,7,8.0,9.0


In [106]:
Count_null = df_m.isnull().sum().sum()

In [109]:
num_cell = np.product(df_m.shape)

In [110]:
num_cellw

9

In [98]:
df_row = df_m.dropna(axis = 0)

In [99]:
df_row

,X,Y,Z
2,7,8.0,9.0


In [100]:
df_column = df_m.dropna(axis = 1)

In [101]:
df_column

,X
0,1
1,4
2,7


**__x___X____X_XXX_____X_____X_______X________X_______X_______X**

# Filling the missing values automatically

Another option is to try and fill in the missing values. For this next bit, I'm getting a small sub-section of the NFL data so that it will print well.

In [117]:
# get a small subset of the NFL dataset
subset_fb_data = fb_event_data.loc[:,'EPA':'Season'].head(10)
subset_fb_data 

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,NaN,NaN,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,NaN,NaN,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,NaN,NaN,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,NaN,NaN,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,NaN,NaN,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,NaN,NaN,2009
5,-0.696302,NaN,NaN,0.558929,0.441071,0.578453,0.421547,0.441071,-0.019524,NaN,NaN,2009
6,-0.179149,-0.343085,0.163935,0.578453,0.421547,0.582881,0.417119,0.421547,-0.004427,-0.010456,0.006029,2009
7,-1.119477,NaN,NaN,0.582881,0.417119,0.617544,0.382456,0.417119,-0.034663,NaN,NaN,2009
8,-0.021313,NaN,NaN,0.617544,0.382456,0.591489,0.408511,0.382456,0.026054,NaN,NaN,2009
9,-0.215293,-0.756894,0.541602,0.591489,0.408511,0.585405,0.414595,0.591489,-0.006084,-0.024526,0.018442,2009


We can use the Panda's fillna() function to fill in missing values in a dataframe for us. One option we have is to specify what we want the NaN values to be replaced with. Here, I'm saying that I would like to replace all the NaN values with 0.

In [118]:
# replace all NaN's with 0

subset_fb_data.fillna(0) # fillna() is used to replace all the NaN value with specified values

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,0.000000,0.000000,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,0.000000,0.000000,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,0.000000,0.000000,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.000000,0.000000,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,0.000000,0.000000,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,0.000000,0.000000,2009
5,-0.696302,0.000000,0.000000,0.558929,0.441071,0.578453,0.421547,0.441071,-0.019524,0.000000,0.000000,2009
6,-0.179149,-0.343085,0.163935,0.578453,0.421547,0.582881,0.417119,0.421547,-0.004427,-0.010456,0.006029,2009
7,-1.119477,0.000000,0.000000,0.582881,0.417119,0.617544,0.382456,0.417119,-0.034663,0.000000,0.000000,2009
8,-0.021313,0.000000,0.000000,0.617544,0.382456,0.591489,0.408511,0.382456,0.026054,0.000000,0.000000,2009
9,-0.215293,-0.756894,0.541602,0.591489,0.408511,0.585405,0.414595,0.591489,-0.006084,-0.024526,0.018442,2009


I could also be a bit more savvy and replace missing values with whatever value comes directly after it in the same column. (This makes a lot of sense for datasets where the observations have some sort of logical order to them.)

In [119]:
# if you wnat to replace all the NaN's with next row value of the same  column then
# use bfill method
subset_fb_data.fillna(method='bfill',axis=0).fillna(0) 

,EPA,airEPA,yacEPA,Home_WP_pre,Away_WP_pre,Home_WP_post,Away_WP_post,Win_Prob,WPA,airWPA,yacWPA,Season
0,2.014474,-1.068169,1.146076,0.485675,0.514325,0.546433,0.453567,0.485675,0.060758,-0.032244,0.036899,2009
1,0.077907,-1.068169,1.146076,0.546433,0.453567,0.551088,0.448912,0.546433,0.004655,-0.032244,0.036899,2009
2,-1.402760,3.318841,-5.031425,0.551088,0.448912,0.510793,0.489207,0.551088,-0.040295,0.106663,-0.156239,2009
3,-1.712583,3.318841,-5.031425,0.510793,0.489207,0.461217,0.538783,0.510793,-0.049576,0.106663,-0.156239,2009
4,2.097796,-0.343085,0.163935,0.461217,0.538783,0.558929,0.441071,0.461217,0.097712,-0.010456,0.006029,2009
5,-0.696302,-0.343085,0.163935,0.558929,0.441071,0.578453,0.421547,0.441071,-0.019524,-0.010456,0.006029,2009
6,-0.179149,-0.343085,0.163935,0.578453,0.421547,0.582881,0.417119,0.421547,-0.004427,-0.010456,0.006029,2009
7,-1.119477,-0.756894,0.541602,0.582881,0.417119,0.617544,0.382456,0.417119,-0.034663,-0.024526,0.018442,2009
8,-0.021313,-0.756894,0.541602,0.617544,0.382456,0.591489,0.408511,0.382456,0.026054,-0.024526,0.018442,2009
9,-0.215293,-0.756894,0.541602,0.591489,0.408511,0.585405,0.414595,0.591489,-0.006084,-0.024526,0.018442,2009
